In [30]:
import PIL.Image

import tensorflow as tf
import numpy as np

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, load_img

from collections import Counter



from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD, Nadam

import tkinter.filedialog as filedialog
import tkinter as tk
import time

from tkinter.filedialog import askopenfilename
from tkinter import *
from tkinter.ttk import *

image_width, image_height = 224, 224
weights_path = r'weights-improvement-05-0.138-0.970.hdf5'

In [31]:
def crop(input_file, height, width):
    img = PIL.Image.open(input_file)
    img_width, img_height = img.size
    crop_img = []
    if img_height%height == 0 :
        i_iter = img_height//height
    else :
        i_iter = img_height//height+1
    
    if img_width%width == 0 :
        j_iter = img_width//width
    else :
        j_iter = img_width//width+1
        
    for i in range(i_iter):
        for j in range(j_iter):
            box = (j*width, i*height, (j+1)*width, (i+1)*height)
            crop_img.append(img.crop(box))
    
    return crop_img

In [32]:
def create_model():
    inceptionV3 = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3))
    
    model = Sequential()
    model.add(inceptionV3)
    model.add(GlobalAveragePooling2D())


    model.add(Dense(512,activation=('relu')))
    model.add(Dropout(0.4))

    model.add(Dense(256,activation=('relu')))
    model.add(Dropout(0.4))

    model.add(Dense(128,activation=('relu')))
    model.add(Dropout(0.4))

    model.add(Dense(64,activation=('relu')))
    model.add(Dropout(0.4))

    model.add(Dense(3,activation=('softmax')))
    
    for layer in model.layers[:-50]:
        layer.trainable = False
        
    model.compile(optimizer=Adam(lr=.00001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = create_model()
model.load_weights(weights_path)

In [37]:
def prediction(path):    
    predict = []

    for index, img in enumerate(crop(path, 224, 224)):
        img_array = img_to_array(img)/255
        img_array = np.expand_dims(img_array, axis=0)
        predict.append(np.argmax(model.predict(img_array), axis=-1)[0])
        print(model.predict(img_array))
        
    predict_dict = dict(Counter(predict))
    
#     print(predict_dict)
    print('Classification : ')
    try:
        print('CLL :', predict_dict[0]/len(predict)*100, '%')
        X = predict_dict[0]/len(predict)*100
    except:
        print('CLL :', '0 %')
        X = 0
    try:
        print('FL :', predict_dict[1]/len(predict)*100, '%')
        Y = predict_dict[1]/len(predict)*100
    except:
        print('FL :', '0 %')
        Y = 0
    try:    
        print('MCL :', predict_dict[2]/len(predict)*100, '%')
        Z = predict_dict[2]/len(predict)*100
    except:
        print('MCL :', '0 %')
        Z = 0
        
    nl = '\n'
    
    print(path)
    print(predict)
    prediction_result["text"]=f"Result : {nl} CLL : {round(X,2)}% {nl}FL : {round(Y,2)}% {nl} MCL : {round(Z,2)}%"

In [ ]:
nl = '\n'
X = "??"
Y = "??"
Z = "??"

window = tk.Tk()

def bar():
    progress['value'] = 20
    window.update_idletasks()
    time.sleep(1)
  
    progress['value'] = 40
    window.update_idletasks()
    time.sleep(1)
  
    progress['value'] = 50
    window.update_idletasks()
    time.sleep(1)
  
    progress['value'] = 60
    window.update_idletasks()
    time.sleep(1)
  
    progress['value'] = 80
    window.update_idletasks()
    time.sleep(1)
    progress['value'] = 100
    
    
def input():
    filename = filedialog.askopenfilename(parent=window,title='Please select a file')
    if filename:
        bar()
        prediction(filename)


window.minsize(500, 300)
window.columnconfigure(0, weight=1, minsize=100)
window.columnconfigure(1, weight=3, minsize=250)
window.columnconfigure(2, weight=1, minsize=100)


window.rowconfigure([0, 1, 2], minsize=100)
window.title("lymphoma classification prediction")
               

input_path = tk.Label(text="Input File Path:")
input_path.grid(row=0, column=0, padx=5)
input_entry = tk.Entry(text="", width=40)
input_entry.grid(row=0, column=1, padx=5)
browse1 = tk.Button(text="Select and lanch", command=input)
browse1.grid(row=0, column=2, padx=5)


progress = Progressbar(orient = HORIZONTAL, length = 400, mode = 'determinate')
progress.grid(row=1, columnspan=3)


prediction_result = tk.Label(text=f"CLL : {X}% {nl}FL : {Y}% {nl} MCL : {Z}%")
prediction_result.grid(row=2, column=1, pady=5)

window.mainloop()

[[0.02611853 0.09428026 0.8796012 ]]
[[6.2416314e-07 1.4745599e-06 9.9999785e-01]]
[[4.5420738e-06 2.1147327e-05 9.9997425e-01]]
[[3.6346689e-06 1.2237072e-05 9.9998415e-01]]
[[6.6424855e-06 2.9277262e-05 9.9996412e-01]]
[[1.7574255e-04 5.8528531e-04 9.9923897e-01]]
[[8.7589593e-05 1.3701689e-04 9.9977535e-01]]
[[9.5546078e-05 1.7778206e-04 9.9972671e-01]]
[[7.423574e-05 4.523873e-04 9.994734e-01]]
[[1.8026218e-09 8.9662997e-09 1.0000000e+00]]
[[4.5659076e-04 9.3925511e-04 9.9860412e-01]]
[[2.6172968e-06 1.2450218e-05 9.9998498e-01]]
[[5.9640931e-04 1.2243927e-03 9.9817920e-01]]
[[8.9079340e-06 1.3473406e-05 9.9997759e-01]]
[[1.9486279e-04 9.0038183e-04 9.9890471e-01]]
[[1.9060368e-06 5.5555206e-06 9.9999249e-01]]
[[5.7616813e-07 2.4913393e-06 9.9999690e-01]]
[[7.2545622e-06 2.3816097e-05 9.9996889e-01]]
[[3.2662920e-10 1.4938817e-09 1.0000000e+00]]
[[5.8851794e-05 3.2458626e-04 9.9961650e-01]]
[[1.5641842e-07 4.8137923e-07 9.9999940e-01]]
[[7.4608206e-06 2.0027082e-05 9.9997246e-01]]
